# BJJ 

The purpose of this workbook is to examine relationships between Brazilian Jiu Jitsu Athletes via network analysis.
My goal is to scrape athlete names and records from FloGrappling, add the basic information into a SQLite database, and use this to get all matches for all athletes. 
I will use this to explore networks using the networkx package.

I am expecting to find clusers of athletes based on sex and weight class. I would like to see if I can visualize the top ranked competitor based on the number of athletes they've beaten in the cluster.

In [88]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import pandas as pd
import numpy as np
import re
import sqlite3
import time
from pytictoc import TicToc

# Get all athletes


In [2]:
# Get all athletes
url = 'https://www.flograppling.com/people'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')
#print(soup)

In [3]:
Athletes = pd.DataFrame(columns=['Athlete_Name','Athlete_ID'], index=range(0,1000))

row = 0
for t in soup.find_all('div', class_ = 'grid-container-item grid-container-item__min-width mb-4 ng-star-inserted'):
    Name = t.get_text()
    Athletes.iloc[row,0] = Name
    for p in t('a'):
        ID = p.get('href')
        Athletes.iloc[row,1] = ID
        row = row + 1
Athletes = Athletes.dropna(axis=0,how='all')
Athletes

,Athlete_Name,Athlete_ID
0,"Aaron ""Tex"" Johnson",/people/6600946-aaron-tex-johnson
1,Adam Wardzinski,/people/5950131-adam-wardzinski
2,Alexssandro Sodre,/people/6749113-alexssandro-sodre
3,Ana Carolina Vieira,/people/5950497-ana-carolina-vieira
4,Andre Galvao,/people/5951184-andre-galvao
...,...,...
97,Vagner Rocha,/people/5950392-vagner-rocha
98,Victor Hugo,/people/6566365-victor-hugo
99,"Vinicius ""Trator"" Ferreira",/people/6708867-vinicius-trator-ferreira
100,Vinny Magalhaes,/people/5950917-vinny-magalhaes


# Can I get current record & Age

In [54]:
# Sample Athlete
#url = 'https://www.flograppling.com/people/6566775-craig-jones'
#url = 'https://www.flograppling.com/people/6801756-danielle-kelly'
#url = 'https://www.flograppling.com/people/6801763-haisam-rida'
#url = 'https://www.flograppling.com/people/6578323-cole-abate'
url = 'https://www.flograppling.com/people/6566375-ethan-crelinsten'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

In [79]:
def get_record(athlete_id):
    url = 'https://www.flograppling.com' + athlete_id
    html = urllib.request.urlopen(url).read()
    athlete_parsed_html = BeautifulSoup(html, 'html.parser')
    try:
        current_record_text = athlete_parsed_html('h2')[0].get_text()
        #current_record_text = athlete_parsed_html('h2')[0].get_text()
        current_record_num = current_record_text.split()[0]
        return(current_record_num)
    except:
        print('No Record')   

In [211]:
# Regex experiment
for line in soup('meta'):
    textline = line.get('content')
    if textline is not None:
        #print(textline)
        #print(re.search('\w+\s\d+\W\s\d\d\d\d',textline))
        birthdate = re.search('\w+\s\d+\W\s\d\d\d\d',textline)
        if birthdate is not None:
            print(birthdate[0])
            break

May 14, 1994


In [80]:
def get_age(athlete_id):
    url = 'https://www.flograppling.com' + athlete_id
    html = urllib.request.urlopen(url).read()
    athlete_parsed_html = BeautifulSoup(html, 'html.parser')
    birthdate=[]
    for line in athlete_parsed_html('meta'):
        textline = line.get('content')
        if textline is not None:
            #print(textline)
            birthdate = (re.search('\w+\s\d+\W\s\d\d\d\d',textline))
            if birthdate is not None:
                birthdate = birthdate[0]
                birthdate = birthdate.replace(',','')
                return birthdate
                break


In [81]:
def get_name(athlete_id):
    url = 'https://www.flograppling.com' + athlete_id
    html = urllib.request.urlopen(url).read()
    athlete_parsed_html = BeautifulSoup(html, 'html.parser')
    name = athlete_parsed_html('title')[0].get_text().split('|')[0]
    return name

In [194]:
# Combine
Record_Birth = pd.DataFrame(columns=['Record','Birthdate'], index=range(0,1000))

'https://www.flograppling.com' + Athletes['Athlete_ID'][0]
#Athletes['Athlete_ID'][0].apply(get_age,axis=1)

row = 0
for id in Athletes['Athlete_ID']:
    Record_Birth.iloc[row,0] = get_record(id)
    Record_Birth.iloc[row,1] = get_age(id)
    row = row + 1
    #print(get_age(id))
    #print(get_record(id))
    
Record_Birth = Record_Birth.dropna(axis=0,how='all')
Record_Birth

No Record
No Record
No Record


,Record,Birthdate
0,87-70-1,None
1,180-52-2,None
2,82-28-0,None
3,47-12-0,November 19 1993
4,165-31-2,September 29 1982
...,...,...
97,71-26-3,June 6 1982
98,102-27-0,May 1 1997
99,64-37-0,None
100,37-25-3,None


In [5]:
Athlete_Info = Athletes.join(Record_Birth)
Athlete_Info

NameError: name 'Athletes' is not defined

# Database Work
Open bjj_network database in SQLite
Add Athlete_Info to a table within the database.

In [7]:
# Connect to SQLite Server
con = sqlite3.connect("bjj_network.db")
cur = con.cursor()

In [203]:
# Create a table for athletes
cur.execute("CREATE TABLE Athlete_List(Athlete_Name,Athlete_ID,Current_Record, DOB)")

In [207]:
#cur.executemany("INSERT INTO Athlete_List (Athlete_Name,Athlete_ID,Record,DOB) Values(?,?,?,?)",Athlete_Info)
Athlete_Info.to_sql('Athlete_List', con, if_exists='replace')
con.commit()

In [18]:
# Pull
Athlete_Info = pd.read_sql_query('''SELECT * FROM Athlete_List''',con)

# Table Format
0. Year
1. Date
2. W/L
3. Replay
4. Method
5. Opponent
6. Weight
7. Event

In [216]:
# Sample Athlete
#url = 'https://www.flograppling.com/people/6566775-craig-jones'
url = 'https://www.flograppling.com/people/6801756-danielle-kelly'
#url = 'https://www.flograppling.com/people/6801763-haisam-rida'
#url = 'https://www.flograppling.com/people/6578323-cole-abate'
#url = 'https://www.flograppling.com/people/6566375-ethan-crelinsten'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')
# Loop
Columns = ['Year','Date','Date_Link','W/L','W/L_Link','Replay','Replay_Link','Method','Method_Link','Opponent','Opponent_Link','Weight','Weight_Link','Event','Event_Link']
Athlete_Matches = pd.DataFrame(columns=Columns,index=range(0,1000)) # Premake table
idx=1
rownum = 0
for t in soup('flo-section-table'): # This will get all the tables on the page
    
    Year = t.find('h5').get_text()
    if Year == 'W/L Methods': # Flo has a table that we don't care about called 'W/L Methods'
        break
    #print(t.find('h6'))
    #print('-------') 
    for td in t.find_all('td'): # td is cell info within a table in html
        link_idx = idx + 1 
        #print(td.get_text())
        text = td.get_text() # the text stored in the cell (what is visible on the website)
        if text == 'Submission': # This line is meant to stop the loop at the end of the tables
            break # Note that Flo has a table with win/loss methods & submission win/loss methods at the end
        link = td('a')[0].get('href') # the link associated with that text (if there is one)

        
        Athlete_Matches.iloc[rownum, idx] = text
        Athlete_Matches.iloc[rownum, link_idx] = link
        #print('Column:',idx,'Year:',Year,'Text:',text,'Link:',link) # Print it all so I can check it works
        idx = idx + 2
        
        if idx == 15: # This is the piece that keeps track of what column it is
            idx=1
            Athlete_Matches.iloc[rownum, 0] = Year # Add year right before we switch the row
            rownum = rownum + 1
    #for piece in t:
        #print(piece.find_all('td'))
Athlete_Matches = Athlete_Matches.dropna(axis=0,how='all') # Get rid of all the extra rows
display(Athlete_Matches)


,Year,Date,Date_Link,W/L,W/L_Link,Replay,Replay_Link,Method,Method_Link,Opponent,Opponent_Link,Weight,Weight_Link,Event,Event_Link
0,2023,02/24,,W,,,,Decision,,A. Miura,,---,,2023 One Fight Night 7,/events/10664382
1,2022,03/26,,D,,,,Other,,M. Yamaguchi,,Absolute Division,,2022 One FC X,/events/10667986
2,2021,12/30,,W,,,,Injury,,C. Esparza,,Absolute Division,,2021 Fury Pro 3,/events/10668486
3,2021,12/16,,L,,,,Decision,,A. Yanes,/people/6808816,Absolute Division,,2021 UFC Fight Pass Invitational,/events/10668345
4,2021,09/25,,L,,,/events/7074608/videos?playing=7158673,Decision,/events/7074608/videos?playing=7158673,G. Gundrum,/people/6759766,115lbs,,2021 FloGrappling WNO Championship,/events/7074608
5,2021,09/25,,W,,,/events/7074608/videos?playing=7159702,Heel Hook,/events/7074608/videos?playing=7159702,J. Crane,,115lbs,,2021 FloGrappling WNO Championship,/events/7074608
6,2021,09/25,,L,,,/events/7074608/videos?playing=7159891,Armlock,/events/7074608/videos?playing=7159891,T. Musumeci,/people/5951234,115lbs,,2021 FloGrappling WNO Championship,/events/7074608
7,2021,06/18,,W,,,/events/7050337/videos?playing=7085353,Knee Bar,/events/7050337/videos?playing=7085353,J. Crane,,115lbs,,WNO: Craig Jones vs Tye Ruotolo,/events/7050337
8,2021,02/26,,L,,,/events/6866574/videos?playing=6900569,Decision,/events/6866574/videos?playing=6900569,J. Khan,/people/6593537,---,,WNO: Craig Jones vs Ronaldo Junior,/events/6866574
9,2020,02/24,,L,,,,Other,,N. Sullivan,,Lightweight,,2020 IBJJF San Jose Open No Gi,/events/10668910


In [24]:
def get_matches(athlete_id):
    url = 'https://www.flograppling.com' + athlete_id
    html = urllib.request.urlopen(url).read()
    athlete_parsed_html = BeautifulSoup(html, 'html.parser')
    
    Columns = ['Athlete_ID','Year','Date','Date_Link','W/L','W/L_Link','Replay','Replay_Link','Method','Method_Link','Opponent','Opponent_Link','Weight','Weight_Link','Event','Event_Link']
    Athlete_Matches = pd.DataFrame(columns=Columns,index=range(0,1000)) # Premake table
    
    idx=2
    rownum = 0
    for t in athlete_parsed_html('flo-section-table'): # This will get all the tables on the page
    
        Year = t.find('h5').get_text()
        if Year == 'W/L Methods': # Flo has a table that we don't care about called 'W/L Methods'
            break

        for td in t.find_all('td'): # td is cell info within a table in html
            link_idx = idx + 1 
                    
            text = td.get_text() # the text stored in the cell (what is visible on the website)
            if text == 'Submission': # This line is meant to stop the loop at the end of the tables
                break # Note that Flo has a table with win/loss methods & submission win/loss methods at the end
            link = td('a')[0].get('href') # the link associated with that text (if there is one)
        
            Athlete_Matches.iloc[rownum, idx] = text
            Athlete_Matches.iloc[rownum, link_idx] = link
            idx = idx + 2
        
            if idx == 16: # This is the piece that keeps track of what column it is
                idx=2
                Athlete_Matches.iloc[rownum, 1] = Year # Add year right before we switch the row
                Athlete_Matches.iloc[rownum, 0] = athlete_id
                rownum = rownum + 1

    Athlete_Matches = Athlete_Matches.dropna(axis=0,how='all') # Get rid of all the extra rows
    return Athlete_Matches

In [25]:
get_matches(Athlete_Info['Athlete_ID'][0])

,Athlete_ID,Year,Date,Date_Link,W/L,W/L_Link,Replay,Replay_Link,Method,Method_Link,Opponent,Opponent_Link,Weight,Weight_Link,Event,Event_Link
0,/people/6600946-aaron-tex-johnson,2021,09/25,,W,,,/events/7074608/videos?playing=7157529,DQ,/events/7074608/videos?playing=7157529,O. Sanchez,/people/6571306,Heavyweight,,2021 FloGrappling WNO Championship,/events/7074608
1,/people/6600946-aaron-tex-johnson,2021,09/25,,L,,,/events/7074608/videos?playing=7159211,Decision,/events/7074608/videos?playing=7159211,T. Spriggs,/people/5950976,Heavyweight,,2021 FloGrappling WNO Championship,/events/7074608
2,/people/6600946-aaron-tex-johnson,2021,07/09,,L,,,,Decision,,G. Batista,/people/5952220,Absolute Division,,Fight to Win 176,/events/7009421
3,/people/6600946-aaron-tex-johnson,2021,06/18,,W,,,/events/7009409/videos?playing=7086209,Decision,/events/7009409/videos?playing=7086209,L. Barbosa,/people/5951235,---,,Fight to Win 174,/events/7009409
4,/people/6600946-aaron-tex-johnson,2021,05/15,,W,,,/events/6921998/videos?playing=7043301,Other,/events/6921998/videos?playing=7043301,J. Hansen,/people/6749112,Ultra Heavyweight,,2021 Pan IBJJF Jiu-Jitsu No-Gi Championship,/events/6921998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,/people/6600946-aaron-tex-johnson,2014,03/12,,L,,,,Reverse Triangle,,G. Ryan,/people/5950166,Absolute Division,,2014 Pan Jiu-Jitsu IBJJF Championship,
154,/people/6600946-aaron-tex-johnson,2014,03/12,,L,,,,OT,,K. Duarte,/people/6451325,---,,2014 Pan Jiu-Jitsu IBJJF Championship,
155,/people/6600946-aaron-tex-johnson,2014,03/12,,W,,,,Points,,G. Ryan,/people/5950166,Absolute Division,,2014 Pan Jiu-Jitsu IBJJF Championship,
156,/people/6600946-aaron-tex-johnson,2014,03/12,,L,,,,Points,,D. Jackson,/people/5951233,Absolute Division,,2014 Pan Jiu-Jitsu IBJJF Championship,


In [33]:
# Initialize an empty list to store DataFrames
all_dataframes = []

# Iterate through the list of athlete IDs
for id in Athlete_Info['Athlete_ID']:
    matches = get_matches(id)
    time.sleep(2)
    print('Working on: ',id)
    
    # Append the DataFrame to the list
    all_dataframes.append(matches)

master_df = pd.concat(all_dataframes, ignore_index=True)
master_df.tail()

Working on " /people/6600946-aaron-tex-johnson
Working on " /people/5950131-adam-wardzinski
Working on " /people/6749113-alexssandro-sodre
Working on " /people/5950497-ana-carolina-vieira
Working on " /people/5951184-andre-galvao
Working on " /people/6749127-andrew-tackett
Working on " /people/5950723-andrew-wiltse
Working on " /people/6789937-andy-murasaki
Working on " /people/6749169-ashley-williams
Working on " /people/5950327-bianca-basilio
Working on " /people/5951190-caio-terra
Working on " /people/6578323-cole-abate
Working on " /people/6749128-cole-franson
Working on " /people/6566775-craig-jones
Working on " /people/6801756-danielle-kelly
Working on " /people/5950334-dante-leon
Working on " /people/6749155-david-garmo
Working on " /people/5951106-dillon-danis
Working on " /people/6749106-edwin-junny-ocasio
Working on " /people/6749300-elder-cruz
Working on " /people/6614340-elisabeth-clay
Working on " /people/6566375-ethan-crelinsten
Working on " /people/5947288-felipe-pena
Wo

,Athlete_ID,Year,Date,Date_Link,W/L,W/L_Link,Replay,Replay_Link,Method,Method_Link,Opponent,Opponent_Link,Weight,Weight_Link,Event,Event_Link
9673,/people/6587288-william-tackett,2018,11/03,,W,,,/events/6180209/videos?playing=6262337,Other,/events/6180209/videos?playing=6262337,D. Davis,/people/6801831,-77kgs,,2018 ADCC North American Trials,/events/6180209
9674,/people/6587288-william-tackett,2018,10/01,,L,,,,Kimura,,P. Barch,/people/6749124,-77kgs,,2018 Onnit Invitational 9,/events/10668377
9675,/people/6587288-william-tackett,2018,01/12,,W,,,/events/6020202/videos?playing=6083483,Triangle,/events/6020202/videos?playing=6083483,C. Foster,/people/6810196,---,,Fight 2 Win Pro 58,/events/6020202
9676,/people/6587288-william-tackett,2017,03/03,,W,,,/events/5011069/videos?playing=5758462,Choke,/events/5011069/videos?playing=5758462,K. Amado,/people/6809956,160lbs,,Fight 2 Win Pro 27,/events/5011069
9677,/people/6587288-william-tackett,2014,03/12,,L,,,,Golden Score,,V. Hugo,/people/6566365,---,,2014 Pan Jiu-Jitsu IBJJF Championship,


In [46]:
# Get unique opponent links
unique_opponents = master_df['Opponent_Link'].unique()
unique_opponents = pd.Series(unique_opponents)

In [48]:
# Remove the words from Athlete ID for easier comparison
no_text_ids = []
for id in Athlete_Info['Athlete_ID']:
    no_text_ids.append(id.split('-')[0])
    
no_text_ids = pd.Series(no_text_ids)

In [74]:
# Get the unique opponents not in the originial 100 athlete list
new_opponent_ids= unique_opponents[~unique_opponents.isin(no_text_ids)]
new_opponent_ids = new_opponent_ids.dropna(how='all')
new_opponent_ids = new_opponent_ids[new_opponent_ids.str.contains('/people')]
new_opponent_ids

0       /people/6571306
2       /people/5952220
4       /people/6749112
8       /people/6800513
9       /people/6808299
             ...       
1398    /people/6808489
1399    /people/6809734
1400    /people/6809699
1401    /people/6810196
1402    /people/6809956
Length: 1303, dtype: object

In [84]:
New_Athletes = pd.DataFrame(columns=['Athlete_Name','Athlete_ID','Record','Birthdate'], index=range(0,10000))
row = 0
for id in new_opponent_ids:
    try:
        print('Working on: ',id)
        New_Athletes.iloc[row,0] = get_name(id)
        New_Athletes.iloc[row,1] = id
        New_Athletes.iloc[row,2] = get_record(id)
        New_Athletes.iloc[row,3] = get_age(id)
        row = row + 1
        time.sleep(2)
    except:
        pass
New_Athletes = New_Athletes.dropna(axis=0,how='all')
New_Athletes

Working on:  /people/6571306
Working on:  /people/5952220
Working on:  /people/6749112
Working on:  /people/6800513
Working on:  /people/6808299
Working on:  /people/6808497
Working on:  /people/6810533
Working on:  /people/6747440
Working on:  /people/6809733
Working on:  /people/6749156
Working on:  /people/5950737
Working on:  /people/6749269
Working on:  /people/6809064
Working on:  /people/6801845
Working on:  /people/6566444
Working on:  /people/5951244
Working on:  /people/6750046
Working on:  /people/6801791
Working on:  /people/6801816
Working on:  /people/6801807
Working on:  /people/6749263
Working on:  /people/5950177
Working on:  /people/6808645
Working on:  /people/6749144
Working on:  /people/5950133
Working on:  /people/5951167
Working on:  /people/6811194
Working on:  /people/6800317
Working on:  /people/6808625
Working on:  /people/6749199
Working on:  /people/6749114
Working on:  /people/5950742
Working on:  /people/5952103
Working on:  /people/6761731
Working on:  /

Working on:  /people/6829340
Working on:  /people/6811057
Working on:  /people/6810392
Working on:  /people/6808267
Working on:  /people/6774939
Working on:  /people/6810612
Working on:  /people/6751138
Working on:  /people/6808708
Working on:  /people/6776189
Working on:  /people/6808654
Working on:  /people/6749162
Working on:  /people/6808187
Working on:  /people/6811256
Working on:  /people/6773542
Working on:  /people/6808458
Working on:  /people/6808778
Working on:  /people/5950727
Working on:  /people/7215937
Working on:  /people/6810490
Working on:  /people/6808373
Working on:  /people/5951471
Working on:  /people/6749166
Working on:  /people/6810929
Working on:  /people/6749108
Working on:  /people/6749176
Working on:  /people/6810492
Working on:  /people/6802442
Working on:  /people/6749160
Working on:  /people/6808247
Working on:  /people/6808365
Working on:  /people/5947294
Working on:  /people/5951163
Working on:  /people/6753980
Working on:  /people/5951313
Working on:  /

Working on:  /people/5950103
Working on:  /people/5950174
Working on:  /people/6809120
Working on:  /people/6810148
Working on:  /people/6810525
Working on:  /people/6810097
Working on:  /people/6808274
Working on:  /people/6809501
Working on:  /people/6809219
Working on:  /people/5950561
Working on:  /people/6809277
Working on:  /people/5951165
Working on:  /people/5951166
Working on:  /people/5952141
Working on:  /people/5951243
Working on:  /people/5950915
Working on:  /people/6776087
Working on:  /people/6808314
Working on:  /people/6749183
Working on:  /people/6889169
Working on:  /people/6810948
Working on:  /people/6801898
Working on:  /people/6749177
Working on:  /people/6801787
Working on:  /people/5951571
Working on:  /people/6889175
Working on:  /people/6811117
Working on:  /people/6808397
Working on:  /people/6808385
Working on:  /people/6566370
Working on:  /people/6774943
Working on:  /people/6808453
Working on:  /people/6811242
Working on:  /people/6810261
Working on:  /

Working on:  /people/6810034
Working on:  /people/6809452
Working on:  /people/6810912
Working on:  /people/6808518
Working on:  /people/5951496
Working on:  /people/6810903
Working on:  /people/6808714
Working on:  /people/6810179
Working on:  /people/6810136
Working on:  /people/6811193
Working on:  /people/6749289
Working on:  /people/6802839
Working on:  /people/6809185
Working on:  /people/6808937
Working on:  /people/6749110
Working on:  /people/6750041
Working on:  /people/6808980
Working on:  /people/6810333
Working on:  /people/6810255
Working on:  /people/6809375
Working on:  /people/6808447
Working on:  /people/6749285
Working on:  /people/6809806
Working on:  /people/5950090
Working on:  /people/6761613
Working on:  /people/5950981
Working on:  /people/6802534
Working on:  /people/6809416
Working on:  /people/5951473
Working on:  /people/6808651
Working on:  /people/6801867
Working on:  /people/6809454
Working on:  /people/6809059
Working on:  /people/6808330
Working on:  /

Working on:  /people/5951132
Working on:  /people/6808305
Working on:  /people/6808817
Working on:  /people/6809423
Working on:  /people/5952102
Working on:  /people/6801877
Working on:  /people/6809824
Working on:  /people/6770004
Working on:  /people/6823359
Working on:  /people/6809049
Working on:  /people/6809703
Working on:  /people/6811184
Working on:  /people/6810670
Working on:  /people/6808513
Working on:  /people/6810838
Working on:  /people/6811246
Working on:  /people/6809485
Working on:  /people/6801813
Working on:  /people/6811154
Working on:  /people/6809425
Working on:  /people/5951424
Working on:  /people/6810636
Working on:  /people/6823369
Working on:  /people/6808819
Working on:  /people/6808699
Working on:  /people/6811210
Working on:  /people/6810201
Working on:  /people/6749212
Working on:  /people/6749290
Working on:  /people/6750078
Working on:  /people/6801890
Working on:  /people/6802448
Working on:  /people/6782699
Working on:  /people/6809223
Working on:  /

,Athlete_Name,Athlete_ID,Record,Birthdate
0,Orlando Sanchez,/people/6571306,13-9-0,None
1,Gustavo Batista,/people/5952220,114-20-0,None
2,John Hansen,/people/6749112,21-16-0,None
3,Giancarlo Bodoni,/people/6800513,41-22-1,None
4,Pedro Palhares,/people/6808299,21-27-0,None
...,...,...,...,...
1289,Michael John,/people/6808489,3-2-0,None
1290,Danny Gutierrez,/people/6809734,0-1-0,None
1291,Joshua Hotta,/people/6809699,0-1-0,None
1292,Cyrus Foster,/people/6810196,0-1-0,None


In [85]:
# Connect to sql database & add to existing table
con = sqlite3.connect("bjj_network.db")
cur = con.cursor()
New_Athletes.to_sql('Athlete_List', con, if_exists='append')
con.commit()

In [118]:
# Pull the combined data
Athlete_List = pd.read_sql_query('''SELECT * FROM Athlete_List''',con)
Athlete_List.tail()

,index,Athlete_Name,Athlete_ID,Record,Birthdate
1391,1289,Michael John,/people/6808489,3-2-0,None
1392,1290,Danny Gutierrez,/people/6809734,0-1-0,None
1393,1291,Joshua Hotta,/people/6809699,0-1-0,None
1394,1292,Cyrus Foster,/people/6810196,0-1-0,None
1395,1293,Kobe Amado,/people/6809956,0-1-0,None


In [120]:
# There are some rows in this dataframe that really shouldn't be there.
# I'm going to remove based on a few factors, including "replay"
# Being longer than 40 characters
# Including ":"
Athlete_List = Athlete_List[Athlete_List['Athlete_Name'].str.len()<=40]
Athlete_List = Athlete_List[Athlete_List['Athlete_Name'].str.contains('Replay') == False]
Athlete_List = Athlete_List[Athlete_List['Athlete_Name'].str.contains(':') == False]
Athlete_List = Athlete_List[Athlete_List['Athlete_Name'].str.contains('=') == False]

In [123]:
Athlete_List['Athlete_Name'].str.len().max()
Athlete_List.shape

(1387, 5)

In [124]:
# Now, can we run through this incredibly large dataset to get all matches for the nearly 1400 athletes total I have.
t = TicToc()
# Initialize an empty list to store DataFrames
dataframe_holder = []
t.tic()
# Iterate through the list of athlete IDs
for id in Athlete_List['Athlete_ID']:
    matches = get_matches(id)
    time.sleep(2)
    print('Working on: ',id)
    
    # Append the DataFrame to the list
    dataframe_holder.append(matches)

t.toc()    
All_Matches = pd.concat(dataframe_holder, ignore_index=True)
All_Matches.tail()

Working on:  /people/6600946-aaron-tex-johnson
Working on:  /people/5950131-adam-wardzinski
Working on:  /people/6749113-alexssandro-sodre
Working on:  /people/5950497-ana-carolina-vieira
Working on:  /people/5951184-andre-galvao
Working on:  /people/6749127-andrew-tackett
Working on:  /people/5950723-andrew-wiltse
Working on:  /people/6789937-andy-murasaki
Working on:  /people/6749169-ashley-williams
Working on:  /people/5950327-bianca-basilio
Working on:  /people/5951190-caio-terra
Working on:  /people/6578323-cole-abate
Working on:  /people/6749128-cole-franson
Working on:  /people/6566775-craig-jones
Working on:  /people/6801756-danielle-kelly
Working on:  /people/5950334-dante-leon
Working on:  /people/6749155-david-garmo
Working on:  /people/5951106-dillon-danis
Working on:  /people/6749106-edwin-junny-ocasio
Working on:  /people/6749300-elder-cruz
Working on:  /people/6614340-elisabeth-clay
Working on:  /people/6566375-ethan-crelinsten
Working on:  /people/5947288-felipe-pena
Wo

Working on:  /people/6749298
Working on:  /people/6787268
Working on:  /people/5950146
Working on:  /people/6808548
Working on:  /people/6808463
Working on:  /people/6811195
Working on:  /people/6801693
Working on:  /people/6810362
Working on:  /people/6809362
Working on:  /people/6810906
Working on:  /people/5950542
Working on:  /people/6808286
Working on:  /people/5950368
Working on:  /people/6808649
Working on:  /people/6809131
Working on:  /people/6808884
Working on:  /people/5952184
Working on:  /people/6808282
Working on:  /people/5951403
Working on:  /people/6809132
Working on:  /people/5950733
Working on:  /people/5950983
Working on:  /people/6811251
Working on:  /people/6610992
Working on:  /people/6761778
Working on:  /people/5950229
Working on:  /people/6810545
Working on:  /people/6808437
Working on:  /people/6808977
Working on:  /people/6749273
Working on:  /people/6808264
Working on:  /people/6761779
Working on:  /people/6811108
Working on:  /people/6829328
Working on:  /

Working on:  /people/5951492
Working on:  /people/5950188
Working on:  /people/5951234
Working on:  /people/6810766
Working on:  /people/6810633
Working on:  /people/6808907
Working on:  /people/6809746
Working on:  /people/6810499
Working on:  /people/5950189
Working on:  /people/6749284
Working on:  /people/6749192
Working on:  /people/6809875
Working on:  /people/6809045
Working on:  /people/5950335
Working on:  /people/6809595
Working on:  /people/6809085
Working on:  /people/5950231
Working on:  /people/6801809
Working on:  /people/5950986
Working on:  /people/6801866
Working on:  /people/5950091
Working on:  /people/6749181
Working on:  /people/6749132
Working on:  /people/6808302
Working on:  /people/6751022
Working on:  /people/6808643
Working on:  /people/6749258
Working on:  /people/6808450
Working on:  /people/6749237
Working on:  /people/6776308
Working on:  /people/5951168
Working on:  /people/6810114
Working on:  /people/5950404
Working on:  /people/6809410
Working on:  /

Working on:  /people/6749277
Working on:  /people/5950175
Working on:  /people/5951140
Working on:  /people/6811227
Working on:  /people/6809492
Working on:  /people/6611008
Working on:  /people/5951126
Working on:  /people/5950886
Working on:  /people/6808250
Working on:  /people/6809491
Working on:  /people/6801764
Working on:  /people/6809494
Working on:  /people/6749152
Working on:  /people/6801834
Working on:  /people/5951232
Working on:  /people/6810289
Working on:  /people/5947297
Working on:  /people/6811215
Working on:  /people/6808369
Working on:  /people/6809237
Working on:  /people/6808560
Working on:  /people/5950178
Working on:  /people/5950159
Working on:  /people/6811219
Working on:  /people/6808818
Working on:  /people/6823121
Working on:  /people/5950077
Working on:  /people/6809141
Working on:  /people/6808395
Working on:  /people/6809026
Working on:  /people/6750032
Working on:  /people/7050579
Working on:  /people/6787264
Working on:  /people/6749189
Working on:  /

Working on:  /people/6809233
Working on:  /people/6810071
Working on:  /people/6809297
Working on:  /people/6829270
Working on:  /people/6808813
Working on:  /people/6808446
Working on:  /people/6808635
Working on:  /people/6808429
Working on:  /people/6808291
Working on:  /people/6810100
Working on:  /people/6802527
Working on:  /people/6809516
Working on:  /people/6810908
Working on:  /people/6808989
Working on:  /people/6808947
Working on:  /people/6808786
Working on:  /people/6802576
Working on:  /people/5950134
Working on:  /people/5950092
Working on:  /people/6808343
Working on:  /people/6809255
Working on:  /people/6809948
Working on:  /people/5951437
Working on:  /people/6808524
Working on:  /people/6808590
Working on:  /people/6818501
Working on:  /people/6808790
Working on:  /people/6808686
Working on:  /people/6801701
Working on:  /people/6808471
Working on:  /people/6749295
Working on:  /people/6808358
Working on:  /people/6802450
Working on:  /people/6810133
Working on:  /

Working on:  /people/6808724
Working on:  /people/6809242
Working on:  /people/6801910
Working on:  /people/6808269
Working on:  /people/6809434
Working on:  /people/6810632
Working on:  /people/6809275
Working on:  /people/6808723
Working on:  /people/6810259
Working on:  /people/6801730
Working on:  /people/6810909
Working on:  /people/6811220
Working on:  /people/6811279
Working on:  /people/5950726
Working on:  /people/6801896
Working on:  /people/6809752
Working on:  /people/6810011
Working on:  /people/6808489
Working on:  /people/6809734
Working on:  /people/6809699
Working on:  /people/6810196
Working on:  /people/6809956
Elapsed time is 7076.575104 seconds.


,Athlete_ID,Year,Date,Date_Link,W/L,W/L_Link,Replay,Replay_Link,Method,Method_Link,Opponent,Opponent_Link,Weight,Weight_Link,Event,Event_Link
38480,/people/6808489,2017,07/14,,L,,,/events/5011601/videos?playing=5858694,RNC,/events/5011601/videos?playing=5858694,G. Soto,/people/6808417,185lbs,,Fight 2 Win Pro 40,/events/5011601
38481,/people/6809734,2019,06/21,,L,,,/events/6363324/videos?playing=6528218,Triangle,/events/6363324/videos?playing=6528218,W. Tackett,/people/6587288,---,,2019 Third Coast Grappling Championship 2,/events/6363324
38482,/people/6809699,2019,02/09,,L,,,/events/6254917/videos?playing=6353630,Other,/events/6254917/videos?playing=6353630,W. Tackett,/people/6587288,-77kgs,,2019 ADCC West Coast Trials,/events/6254917
38483,/people/6810196,2018,01/12,,L,,,/events/6020202/videos?playing=6083483,Triangle,/events/6020202/videos?playing=6083483,W. Tackett,/people/6587288,---,,Fight 2 Win Pro 58,/events/6020202
38484,/people/6809956,2017,03/03,,L,,,/events/5011069/videos?playing=5758462,Choke,/events/5011069/videos?playing=5758462,W. Tackett,/people/6587288,160lbs,,Fight 2 Win Pro 27,/events/5011069


In [125]:
All_Matches.shape

(38485, 16)

In [127]:
# Connect to sql database & add to new table
con = sqlite3.connect("bjj_network.db")
cur = con.cursor()
All_Matches.to_sql('Athlete_Matches', con, if_exists='replace')
con.commit()